In [104]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
import re
import json
from tqdm.notebook import tqdm
from datetime import datetime
from dateutil.relativedelta import relativedelta
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
import urllib
import os , sys
import psycopg2



In [105]:
options = Options()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--disable-extensions")
options.add_argument("--proxy-server='direct://'")
options.add_argument("--proxy-bypass-list=*")
options.add_argument("--start-maximized")
options.add_argument('--headless')
options.add_argument('--disable-gpu')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--no-sandbox')
options.add_argument("--enable-javascript")
options.add_argument('--ignore-certificate-errors')
options.add_argument('window-size=1800,1000')
options.add_argument("--incognito")

In [106]:
def check_XPATH_exist(driver, xPATH):
    try:
        driver.find_element(By.XPATH, xPATH)
        return True
    except:
        return False

In [107]:
driver = webdriver.Chrome(service=Service('/home/ubuntu/chromedriver'), options=options)

In [108]:
h=pd.read_excel('Amazon Scraping.xlsx')

In [114]:
data=pd.DataFrame()
data['product_title']=''
data['product_image_url']=''
data['product_details']=''
data['product_price']=''

In [ ]:
t=0
for i in range(h.shape[0]):
    driver.get(f"https://www.amazon.{h['country'][i]}/dp/{h['Asin'][i]}")
    if check_XPATH_exist(driver,'//*[@id="productTitle"]'):
        title=driver.find_element(By.XPATH,'//*[@id="productTitle"]').text 
        if check_XPATH_exist(driver,'//*[@class="a-size-base a-color-price a-color-price"]'):
            price=driver.find_element(By.XPATH,'//*[@class="a-size-base a-color-price a-color-price"]').text
        elif check_XPATH_exist(driver,'//*[@class="a-price aok-align-center reinventPricePriceToPayMargin priceToPay"]'):
            price=driver.find_element(By.XPATH,'//*[@class="a-price aok-align-center reinventPricePriceToPayMargin priceToPay"]').text
        elif check_XPATH_exist(driver,'//*[@class="a-color-base"]'):
            price=driver.find_element(By.XPATH,'//*[@class="a-color-base"]').text
        elif check_XPATH_exist(driver,'//*[@class="a-size-base a-color-secondary"]'):
            price=driver.find_element(By.XPATH,'//*[@class="a-size-base a-color-secondary"]').text 
        if check_XPATH_exist(driver,'//*[@class="a-unordered-list a-nostyle a-vertical a-spacing-none detail-bullet-list"]'):
            details=driver.find_element(By.XPATH,'//*[@class="a-unordered-list a-nostyle a-vertical a-spacing-none detail-bullet-list"]').text 
        if check_XPATH_exist(driver,'//img[@class="a-dynamic-image image-stretch-horizontal frontImage"]'):
            image_element = driver.find_element(By.XPATH,'''//img[@class="a-dynamic-image image-stretch-horizontal frontImage"]''')
            url= image_element.get_attribute("src")
        elif check_XPATH_exist(driver,'//img[@class="a-dynamic-image a-stretch-horizontal"]'):
            image_element = driver.find_element(By.XPATH,'''//img[@class="a-dynamic-image a-stretch-horizontal"]''')
            url= image_element.get_attribute("src")
            

        data.loc[i,'product_title']=title
        data.loc[i,'product_price']=price
        data.loc[i,'product_details']=details
        data.loc[i,'product_image_url']=url
    
                
    else:
        print('Error 404')
    if i % 100 != 0:
        d+=time.process_time()
        t=d-t
    elif i % 100 == 0:
        d=0
        d+=time.process_time()
        print(f"Execution Time: {t} seconds")
        t=d
    elif i==h.shape[0]:
        print(f"Execution Time: {t} seconds")

        
    

In [116]:
data['product_details']=data['product_details'].apply(lambda x:x.replace('\n',', '))

In [117]:
data

,product_title,product_image_url,product_details,product_price
2,Old Spice Rasur Creme - 70 G (original) - Pack...,https://m.media-amazon.com/images/I/41xHbjdSXJ...,Auslaufartikel (Produktion durch Hersteller ei...,8\n00€
16,Short Story: Violoncello und Klavier. (Cello-B...,https://m.media-amazon.com/images/I/41fD+K43Lc...,"Herausgeber : SCHOTT MUSIC GmbH & Co KG, Mainz...","ab 11,50 €"
17,Short Story: cello and piano.,https://m.media-amazon.com/images/I/41fD+K43Lc...,"Éditeur : Schott (1 janvier 2000), Langue : Fr...","à partir de 23,94 €"
20,Concerto grosso g-Moll: Weihnachtskonzert. op....,https://m.media-amazon.com/images/I/51sNFyf2t6...,"Herausgeber : SCHOTT MUSIC GmbH & Co KG, Mainz...","18,00 €"
21,Concerto grosso G Minor: Christmas Concerto. o...,https://m.media-amazon.com/images/I/51sNFyf2t6...,"Éditeur : Schott (1 janvier 2000), Langue : Fr...","30,99 €"
...,...,...,...,...
847,Stucke(3) op.142 orgue,https://m.media-amazon.com/images/I/51jcnDzKub...,"Éditeur : BREITKOPF (1 janvier 2000), Langue :...","18,15 €"
894,L'ARLESIENNE-SUITE NR. 2 ORCHESTRE,https://m.media-amazon.com/images/I/71NgDgoW-k...,"Herausgeber : BREITKOPF (1. Januar 2000), Spra...","43,60 €"
895,L'arlesienne-suite nr. 2 orchestre,https://m.media-amazon.com/images/I/41ELX18MMV...,"Éditeur : BREITKOPF (1 janvier 2000), Langue :...","52,13 €"
910,EIN DEUTSCHES REQUIEM 2 KLAV,https://m.media-amazon.com/images/I/71U21ngbbv...,"Herausgeber : BREITKOPF (1. Januar 2000), Spra...","92,00 €"


In [118]:
data.reset_index(inplace=True,drop=True)

In [119]:


amz = {}
for j, row in data.iterrows():
    ldata = {
        "product_title": row["product_title"],
        "url": row["product_image_url"],
        "product_details": row["product_details"],
        "product_price": row["product_price"],
    }
    id = j
    
    if id not in amz:
        amz[id] = []
    
    amz[id].append(ldata)


In [120]:
#creating Json file
file_path = "solution.json"

with open(file_path, 'w') as json_file:
    json.dump(amz, json_file,indent=4)

print(f"JSON file created: {file_path}")


JSON file created: solution.json


In [62]:
conn = psycopg2.connect(
    host='localhost',
    port='5432',
    database='jais',
    user='postgres',
    password='1234'
)
cursor = conn.cursor()


In [63]:
create_table_query = """
CREATE TABLE IF NOT EXISTS dataamazon (
    id SERIAL PRIMARY KEY,
    product_title VARCHAR(255),
    product_price VARCHAR(255),
    product_details TEXT,
    product_image_url VARCHAR(255)
)
"""
cursor.execute(create_table_query)



In [64]:
data_tuples = [tuple(x) for x in data.values]


In [65]:
insert_query = """
INSERT INTO dataamazon (product_title, product_price, product_details, product_image_url)
VALUES (%s, %s, %s, %s)
"""


In [66]:
cursor.executemany(insert_query, data_tuples)


In [67]:
conn.commit()
print('data ingested')

# Close the cursor and the database connection
cursor.close()
conn.close()


data ingested


In [121]:
data

,product_title,product_image_url,product_details,product_price
0,Old Spice Rasur Creme - 70 G (original) - Pack...,https://m.media-amazon.com/images/I/41xHbjdSXJ...,Auslaufartikel (Produktion durch Hersteller ei...,8\n00€
1,Short Story: Violoncello und Klavier. (Cello-B...,https://m.media-amazon.com/images/I/41fD+K43Lc...,"Herausgeber : SCHOTT MUSIC GmbH & Co KG, Mainz...","ab 11,50 €"
2,Short Story: cello and piano.,https://m.media-amazon.com/images/I/41fD+K43Lc...,"Éditeur : Schott (1 janvier 2000), Langue : Fr...","à partir de 23,94 €"
3,Concerto grosso g-Moll: Weihnachtskonzert. op....,https://m.media-amazon.com/images/I/51sNFyf2t6...,"Herausgeber : SCHOTT MUSIC GmbH & Co KG, Mainz...","18,00 €"
4,Concerto grosso G Minor: Christmas Concerto. o...,https://m.media-amazon.com/images/I/51sNFyf2t6...,"Éditeur : Schott (1 janvier 2000), Langue : Fr...","30,99 €"
...,...,...,...,...
88,Stucke(3) op.142 orgue,https://m.media-amazon.com/images/I/51jcnDzKub...,"Éditeur : BREITKOPF (1 janvier 2000), Langue :...","18,15 €"
89,L'ARLESIENNE-SUITE NR. 2 ORCHESTRE,https://m.media-amazon.com/images/I/71NgDgoW-k...,"Herausgeber : BREITKOPF (1. Januar 2000), Spra...","43,60 €"
90,L'arlesienne-suite nr. 2 orchestre,https://m.media-amazon.com/images/I/41ELX18MMV...,"Éditeur : BREITKOPF (1 janvier 2000), Langue :...","52,13 €"
91,EIN DEUTSCHES REQUIEM 2 KLAV,https://m.media-amazon.com/images/I/71U21ngbbv...,"Herausgeber : BREITKOPF (1. Januar 2000), Spra...","92,00 €"


In [122]:
from elasticsearch import Elasticsearch
es = Elasticsearch(hosts=["https://localhost:9200"], basic_auth=['elastic', '6TsTDCFqEuLyR8UdykMi'], verify_certs=False)
print(es.info())

{'name': 'DESKTOP-92USQB8', 'cluster_name': 'elasticsearch', 'cluster_uuid': 'v071lW7iRCGj6iQrac2Vsw', 'version': {'number': '8.7.0', 'build_flavor': 'default', 'build_type': 'zip', 'build_hash': '09520b59b6bc1057340b55750186466ea715e30e', 'build_date': '2023-03-27T16:31:09.816451435Z', 'build_snapshot': False, 'lucene_version': '9.5.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


C:\Users\vkray\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\elasticsearch\_sync\client\__init__.py:395: SecurityWarning: Connecting to 'https://localhost:9200' using TLS with verify_certs=False is insecure
  _transport = transport_class(


In [124]:
from elasticsearch import Elasticsearch
import pandas as pd
import json

es = Elasticsearch(['https://127.0.0.1:9200'],basic_auth=['elastic', '6TsTDCFqEuLyR8UdykMi'], verify_certs=False)

index_name = "amazdata"



data1 = data.to_dict(orient='records')

for doc in data1:
    es.index(index=index_name, body=doc)


C:\Users\vkray\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\elasticsearch\_sync\client\__init__.py:395: SecurityWarning: Connecting to 'https://127.0.0.1:9200' using TLS with verify_certs=False is insecure
  _transport = transport_class(
C:\Users\vkray\AppData\Local\Temp\ipykernel_10184\275136405.py:14: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.index(index=index_name, body=doc)
